In [1]:
from numba import jit, njit, cuda , prange
import numba
import numpy as np

# registers = [729, 0, 0]
# program = [0,1,5,4,3,0]

register_a = 0
register_b = 1
register_c = 2

# enabling the JIT causes print statements to not print lists as expected
@njit(parallel=True)
def loop_em(start, end, batch_size, program):
    print(program)

    total_range = end - start
    # handle if not exactly divisible
    num_batches = (total_range // batch_size) + 1
    print(f'total_range={total_range} batch_size={batch_size} num_batches={num_batches} start={start} end={end} program = {program}')

    # iterate across each batch
    for batch_index in prange(num_batches):
        # iterate across each iteration of the batch
        print(f'starting batch {batch_index}')
        for in_batch_index in range(batch_size):
            current = start + (batch_index*batch_size) + in_batch_index
            # handle the fact the batch size is not an exact factor of the total range
            if (current <= end):
                # This program is 8 octal operands so the A register needs to be 8 octal digits.
                registers = np.array([current, 0, 0])
                output = []
                resolve_operand = lambda operand: operand if (operand<4) else registers[operand-4]
                # if (i%1000000 == 0):
                #     print(f'at: {current} : {i}/{loop_iterations}')
                
                # print(f'{in_batch_index}:{current}:{end} program: {program} registers: {registers}')
                address_ptr = 0
                while (address_ptr < len(program)):
                    # numba says these are int64
                    operator = program[address_ptr]
                    operand = program[address_ptr+1]
                    next_address_ptr = address_ptr+2
                    # print (f'address: {address_ptr} operator: {operator} operand: {operand} registers: {registers} ')
                    match (operator):
                        case 0: # adv division register_a ~/ 2^comboOperand
                            registers[register_a] = registers[register_a] // 2 ** resolve_operand(operand)
                        case 1: # bxl bitwise XOR (registerB , operand)
                            registers[register_b] = registers[register_b] ^ operand
                        case 2: # bst operand modulo 8
                            registers[register_b] = resolve_operand(operand) % 8
                        case 3: # jnz jump not zero
                            if (registers[register_a] != 0):
                                next_address_ptr =  operand
                        case 4: #bxc bitwise xor reg b, reg c
                            registers[register_b] = registers[register_b] ^ registers[register_c]
                        case 5: # out % modulo 8
                            output.append(resolve_operand(operand) %8)
                        case 6: # BDV integer division on A , stored in B
                            divisor = 2 ** resolve_operand(operand)
                            registers[register_b] = registers[register_a] // divisor
                        case 7: # CDV
                            divisor = 2 ** resolve_operand(operand)
                            registers[register_c] = registers[register_a] // divisor
                        case _:
                            print('oh no')
                            result = -1
                    
                    address_ptr = next_address_ptr
                    # print(f'now at: {address_ptr} output after {output}')
                    # print(f'final registers: {registers} output {np.array(output)} 
                    # This exists because I played with different lengths while experimenting
                    if (len(output) >= 15
                        and output[0]==program[0] 
                        and output[1]==program[1] 
                        and output[2]==program[2] 
                        and output[3]==program[3]
                        and output[4]==program[4]
                        and output[5]==program[5]
                        and output[6]==program[6]
                        and output[7]==program[7]
                        and output[8]==program[8]
                        and output[9]==program[9]
                        and output[10]==program[10]
                        and output[11]==program[11]
                        and output[12]==program[12]
                        and output[13]==program[13]
                        and output[14]==program[14]
                        #and output[15]==program[15]
                        ):
                        # will not print if njit is enabled
                        #print(f'matches {oct(current)} - {output} -{current}')
                        # use with njit
                        print(f'matches {current}')
                        print(output)
                        # from before we put the loop in
                        # return current
                #print(f'{oct(current)} - {output}')
        print(f'finished batch {batch_index}')
        # return output
    print('done');


In [ ]:
%%time
loop_em(int(0o1000000000000000),
        int(0o1000077777777777),
        int(0o0000000100000000),
        np.array([2,4,1,3,7,5,4,7,0,3,1,5,5,5,3,0]))

# 16 digits octal
# 16th digit must be 1 otherwise return is shorter than the program


[2 4 1 3 7 5 4 7 0 3 1 5 5 5 3 0]
total_range=281474976710655 batch_size=68719476736 num_batches=4096 start=0 end=281474976710655 program = <object type:array(int64, 1d, C)>
starting batch 0
starting batch 3756
starting batch 1026
starting batch 2394
starting batch 513
starting batch 2565
starting batch 2052
starting batch 2223
starting batch 684
starting batch 3586
starting batch 342
starting batch 1539
starting batch 2906
starting batch 855
starting batch 3926
starting batch 1197
starting batch 3246
starting batch 171
starting batch 1368
starting batch 3416
starting batch 3076
starting batch 1881
starting batch 2736
starting batch 1710
